In [6]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import pandas as pd
from tqdm import tqdm
import tifffile

ATLAS_DIR = "/media/share5/MYK/ATLAS/mouse"
is_dev = False

if is_dev:
    rl_file = "AllBrainRegions_Dev.csv"
    ann_tif = {
        "E11.5": "annotation_E11pt5_sagittal.tif",
        "E13.5_head": "annotation_E13pt5_head_400x240.tif",
        "E13.5_embryo": "annotation_E13pt5_whole.tif",
        "E15.5_brain": "annotation_E15pt5_sagittal_brain.tif",
        "E15.5_embryo": "annotation_E15pt5_sagittal_whole.tif",
        "E16.5_brain": "annotation_E16pt5_sagittal_head.tif",
        "E16.5_embryo": "annotation_E16pt5_sagittal_whole.tif",
        "E18.5_whole": "annotation_E18pt5_sagittal_whole.tif",
        "E18.5_rot_hemi": "annotation_E18pt5_sagittalRot_hemi.tif",
        "E18.5_rot_whole": "annotation_E18pt5_sagittalRot_whole.tif"
    }
else:
    rl_file = "AllBrainRegions.csv"
    ann_tif = {
        "Adult_hemi_sagittal_whole": "annotation_25_half_sagittal_whole.tif"
    }

df = pd.read_csv(os.path.join(ATLAS_DIR, rl_file))
df.head()

,id,name,acronym,parent_structure_id,depth
0,0,root,root,-1,0
1,1,Basic cell groups and regions,grey,0,1
2,2,Cerebrum,CH,1,2
3,3,Cerebral cortex,CTX,2,3
4,4,Cortical plate,CTXpl,3,4


### Count labels from Annotation Tif

In [7]:
def get_label_counts(ann_tif):
    ann_map = tifffile.imread(ann_tif)
    dd, hh, ww = ann_map.shape
    vol_map = {}
    for zz in tqdm(range(dd)):
        for yy in range(hh):
            for xx in range(ww):
                label = int(ann_map[zz][yy][xx])
                if label:
                    try:
                        vol_map[label] += 1
                    except KeyError:
                        vol_map[label] = 1
    return vol_map

In [8]:
lblc_maps = {}
for key in ann_tif:
    lblc_maps[key] = get_label_counts(os.path.join(ATLAS_DIR, ann_tif[key]))
    print("age: ", key, "len(lblc_maps.keys()): ", len(lblc_maps[key].keys()))
print("lblc_maps.keys(): ", lblc_maps.keys())

100%|█████████████████████████████████████████| 228/228 [00:19<00:00, 11.80it/s]

age:  Adult_hemi_sagittal_whole len(lblc_maps.keys()):  670
lblc_maps.keys():  dict_keys(['Adult_hemi_sagittal_whole'])


### ADD label counts to Dataframe (recursively for parent regions)

In [9]:
def update_df(df, lblc_map, coln):
    err_lbls = []
    for lno in lblc_map.keys():
        counts = lblc_map[lno]
        try:
            df.at[df[df['id'] == lno].index[0], coln] += counts
        except:
            err_lbls.append(lno)
            continue
        
        # apply also to my parents
        p_lbl = curr_lbl = lno
        while True:
            p_lbl = int(df[df['id'] == curr_lbl]['parent_structure_id'])
            if p_lbl == -1:
                break
                
            p_row = df[df['id'] == p_lbl]
            df.at[p_row.index[0], coln] += counts
            curr_lbl = p_lbl

    return df, err_lbls

In [10]:
err_lbls_map = {}
for k in tqdm(lblc_maps.keys()):
    coln = "volume_%s"%k
    df[coln] = 0
    df_new, err_lbls = update_df(df, lblc_maps[k], coln)
    print("# of error labels: ", len(err_lbls))
    err_lbls_map[k] = err_lbls
    
    # add normlized column
    ddf = df_new[coln]
    mcol_norm =(ddf - ddf.min()) / (ddf.max() - ddf.min())
    df_new['%s_norm'%coln] = mcol_norm
df_new.head()

100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.82s/it]

# of error labels:  0


,id,name,acronym,parent_structure_id,depth,volume_Adult_hemi_sagittal_whole,volume_Adult_hemi_sagittal_whole_norm
0,0,root,root,-1,0,16097057,1.000000
1,1,Basic cell groups and regions,grey,0,1,14400507,0.894605
2,2,Cerebrum,CH,1,2,8830913,0.548604
3,3,Cerebral cortex,CTX,2,3,7090956,0.440513
4,4,Cortical plate,CTXpl,3,4,6805980,0.422809


In [12]:
aaa = df_new[df_new['acronym']=='SS']
aaa.head()

,id,name,acronym,parent_structure_id,depth,volume_Adult_hemi_sagittal_whole,volume_Adult_hemi_sagittal_whole_norm
30,30,Somatosensory areas,SS,5,6,1066625,0.066262


In [13]:
# save
rl_file_new = "AllBrainRegions_New.csv"
df_new.to_csv(os.path.join(ATLAS_DIR, rl_file_new), sep=',', index=False)